# PyMem3DG Tutorial 1 - The Very Basic
`Cuncheng Zhu, Christopher T. Lee`

This tutorial provides a brief example for setting up a membrane simulation using PyMem3DG. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

In [11]:
import pymem3dg as dg

## Mesh initialization
PyMem3DG currently provide functions to generate simple meshes, as well as API to read $\texttt{.ply}$ files. For example, we could generate icosphere for closed membrane simualation and existing meshes based on parametrized ellipsoids. 

In [12]:
icoFace, icoVertex = dg.getIcosphere(1, 3)
oblate = "sample_meshes/oblate.ply"
prolate = "sample_meshes/prolate.ply"

## Instantiate the $\texttt{System}$:  $\texttt{Parameters}$ and  $\texttt{Options}$ 

Specification of the struct `Paramters` is largely user-defined. `Parameters` is the structured into sub-structs. For example, the parameters for bending of `Parameters p` is `p.bending`. Please refer to the documentation for detailed explanation of various entries. Below we shows the simulation setup for a homogeneous membrane undergoing osmotic shrinkage, which is a classic solution of the *shape equation* that results in the biconcave shape similar to the red blood cell. Because of the simplicity of the setup, we only specify essential physical quantities such as the bending rigidity, stretching modulus and bulk modulus, assuming the default  and omitting the majority of the parameters. 

The trajectory of the simulations will be solely determined by the initial condition (initial geometry) and parameters. In this tutorial, we will see how initial geometry affects the results. 

In [13]:
p = dg.Parameters()

p.bending.Kbc = 8.22e-5 

p.tension.Ksg = 0.1

p.osmotic.Kv = 0.01
p.osmotic.cam = 0.7 * 3.14 * 0.75 

g_ico = dg.System(icoFace, icoVertex, p)
g_oblate = dg.System(oblate, p)
g_prolate = dg.System(prolate, p)

## Integration and optimization

PyMem3DG provides various evolvers to integrate the membrane dynamics or find mechanical/chemical equalibrium as the energy minimizer. Please refer to the documentation for details and guidelines. For simplicity, the forward euler method, mathematically equivalent to steepest descent method for optimization is used to integrate the three systems. The following integration is pre-runned. Uncomment $\texttt{fe.integrate()}$ to rerun them.

We first to integrate the oblate shape and we expect to form biconcave shape at the equalibrium. To visualize the result using polyscope, please refer to the documentation and a dedicated tutorial for visualization.


In [6]:

fe = dg.Euler(f = g_oblate, dt = 0.5, total_time = 100000, tSave = 1000, tolerance = 1e-5, outputDir = "output/tutorial1/biconcave")
fe.verbosity = 5
fe.integrate()

True

The initial oblate and the final biconcave shape look as follows:

<img src="output/tutorial1/biconcave/initial.png" width="400" height="200">
<img src="output/tutorial1/biconcave/biconcave.png" width="400" height="200">

The colormap represents the mean curvature of the mesh. 
Similarly, we could have the same procedure with initial condition of prolate shape.

In [7]:
fe = dg.Euler(f = g_prolate, dt = 0.1, total_time = 100000, tSave = 1000, tolerance = 3e-5, outputDir = "output/tutorial1/dumbbell")
fe.verbosity = 5
# fe.integrate()

True

The initial prolate and the final dumbbell shape look as follows:

<img src="output/tutorial1/dumbbell/initial.png" width="400" height="200">
<img src="output/tutorial1/dumbbell/dumbbell.png" width="400" height="200">

We can also do this on an generated icosphere. However, because of the symmetry of a icosphere, it appears that we need large osmotic pressure to kick start the deformation. To change the set up of $\texttt{System}$, it is generally safer practice to reinstantiate a new $\texttt{System}$ object. However, because of the simplicity of current setup, the rest of the system does not have complex dependency on attribute changes. Therefore we directly modify the underlying attribute $K_v$ and increase it by fivefold.

In [10]:
g_ico.parameters.osmotic.n *= 5
fe = dg.Euler(f = g_ico, dt = 0.1, total_time = 10000, tSave = 1000, tolerance = 3e-5, outputDir = "output/tutorial1/star")
fe.verbosity = 5
fe.integrate()

True

Notices that the simulation is not ran until the equilibrium but up to $T = 10000$. It is interrupted intentionally due to corrupted mesh quality.
The initial icosphere and intermediary star shape and final snapshot look as follows:

<img src="output/tutorial1/star/initial.png" width="250" height="140">
<img src="output/tutorial1/star/middle.png" width="250" height="140">
<img src="output/tutorial1/star/final.png" width="250" height="140">

Notice that initial icosphere does not have constant mean curvature due to the existence of irregular points. The initial deformation comforms to the discretization defect. With the high curvature deformation, mesh quality deteriorates. To cope with the problem, please proceed to the next tutorial, where we introduce mesh regularization and mutation for adaptive meshing.